# Project 6nomads

## Requirements

In [1]:
# Uncomment below to install some packages which you're missing

In [2]:
# !pip install numpy
# !pip install pandas

In [3]:
# !pip install scipy 
# !pip install scikit-learn  
# !pip install joblib
# !pip install deap update_checker tqdm stopit
# !pip install tpot

In [24]:
#!pip install xgboost
#there are some issues on macos

## Load packages

In [4]:
import numpy as np
import pandas as pd

In [5]:
from tpot import TPOTClassifier

In [58]:
from sklearn.impute import SimpleImputer

In [45]:
from sklearn.manifold import TSNE
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import GaussianRandomProjection

## Prepare environment

### Random state

In [6]:
random_state=42

## Read data

In [7]:
train_pd = pd.read_csv('data/train.csv', header=None)

In [8]:
test_pd = pd.read_csv('data/test.csv', header=None)

### Name the label column

In [9]:
train_pd.rename(columns={65: 'label'}, inplace=True)
test_pd.rename(columns={65: 'label'}, inplace=True)

## First glance at the data

In [10]:
train_pd

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,label
0,-47,0,-5,-27,-11,-19,-2,-33,37,0,...,0,2,-4,-3,4,-23,-68,15,C,0
1,1,1,-2,-3,-1,-3,39,10,0,-1,...,-7,-2,1,6,-2,4,22,2,C,0
2,1,-4,-2,-6,-20,20,-1,-3,20,3,...,14,2,-2,0,23,21,-3,22,A,1
3,10,2,1,-7,-8,12,41,21,-14,3,...,2,1,2,-2,-8,-29,29,7,C,0
4,1,-3,-2,-6,1,0,10,-6,4,0,...,5,0,-3,-8,-11,-2,19,4,C,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8673,5,4,4,6,20,49,123,34,-27,-1,...,7,3,0,-4,8,17,53,15,C,0
8674,16,6,11,2,-14,-35,-8,19,-23,-9,...,-1,47,9,5,-47,-3,3,0,D,2
8675,-1,-1,-1,-4,-4,-6,-8,-3,-8,-2,...,4,0,3,2,-8,22,4,2,B,3
8676,-29,-24,-4,-10,-9,-6,-4,-11,-10,38,...,1,-6,-2,-2,6,5,1,-10,D,2


In [11]:
train_pd.columns.to_series().groupby(train_pd.dtypes).groups

{dtype('int64'): Index([      0,       1,       2,       3,       4,       5,       6,       7,
              8,       9,      10,      11,      12,      13,      14,      15,
             16,      17,      18,      19,      20,      21,      22,      23,
             24,      25,      26,      27,      28,      29,      30,      31,
             32,      33,      34,      35,      36,      37,      38,      39,
             40,      41,      42,      43,      44,      45,      46,      47,
             48,      49,      50,      51,      52,      53,      54,      55,
             56,      57,      58,      59,      60,      61,      62,      63,
        'label'],
       dtype='object'), dtype('O'): Index([64], dtype='object')}

In [12]:
test_pd

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,label
0,-47,0,-5,-27,-11.0,-19.0,-2.0,-33.0,37,0,...,0,2,-4,-3,4,-23,-68,15,A,0
1,1,1,-2,-3,-1.0,-3.0,39.0,10.0,0,-1,...,-7,-2,1,6,-2,4,22,2,C,0
2,1,-4,-2,-6,-20.0,20.0,-1.0,-3.0,20,3,...,14,2,-2,0,23,21,-3,22,B,1
3,10,2,1,-7,-8.0,12.0,41.0,21.0,-14,3,...,2,1,2,-2,-8,-29,29,7,B,0
4,1,-3,-2,-6,1.0,0.0,10.0,-6.0,4,0,...,5,0,-3,-8,-11,-2,19,4,C,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2,1,0,-3,-17.0,-1.0,9.0,3.0,-7,-3,...,2,-2,-4,-4,-52,-41,-5,11,D,3
2996,2,5,7,13,17.0,62.0,5.0,7.0,-21,-12,...,16,9,-6,-5,0,-17,-9,0,D,2
2997,-4,0,1,-1,4.0,56.0,6.0,6.0,9,2,...,-6,0,-2,-1,15,15,-1,-2,C,3
2998,18,-3,1,4,15.0,19.0,-1.0,-1.0,-10,8,...,42,7,1,-3,14,1,-3,-2,B,1


### Label counts

In [20]:
test_pd.label.value_counts()

1    777
2    777
3    734
0    712
Name: label, dtype: int64

In [21]:
train_pd.label.value_counts()

2    2202
0    2177
1    2152
3    2147
Name: label, dtype: int64

## Feature engineering

### Hot-encoding

In [13]:
train_pd = pd.get_dummies(train_pd)

In [14]:
train_pd

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,label,64_A,64_B,64_C,64_D
0,-47,0,-5,-27,-11,-19,-2,-33,37,0,...,-3,4,-23,-68,15,0,0,0,1,0
1,1,1,-2,-3,-1,-3,39,10,0,-1,...,6,-2,4,22,2,0,0,0,1,0
2,1,-4,-2,-6,-20,20,-1,-3,20,3,...,0,23,21,-3,22,1,1,0,0,0
3,10,2,1,-7,-8,12,41,21,-14,3,...,-2,-8,-29,29,7,0,0,0,1,0
4,1,-3,-2,-6,1,0,10,-6,4,0,...,-8,-11,-2,19,4,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8673,5,4,4,6,20,49,123,34,-27,-1,...,-4,8,17,53,15,0,0,0,1,0
8674,16,6,11,2,-14,-35,-8,19,-23,-9,...,5,-47,-3,3,0,2,0,0,0,1
8675,-1,-1,-1,-4,-4,-6,-8,-3,-8,-2,...,2,-8,22,4,2,3,0,1,0,0
8676,-29,-24,-4,-10,-9,-6,-4,-11,-10,38,...,-2,6,5,1,-10,2,0,0,0,1


In [15]:
test_pd = pd.get_dummies(test_pd)

In [16]:
test_pd

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,label,64_A,64_B,64_C,64_D
0,-47,0,-5,-27,-11.0,-19.0,-2.0,-33.0,37,0,...,-3,4,-23,-68,15,0,1,0,0,0
1,1,1,-2,-3,-1.0,-3.0,39.0,10.0,0,-1,...,6,-2,4,22,2,0,0,0,1,0
2,1,-4,-2,-6,-20.0,20.0,-1.0,-3.0,20,3,...,0,23,21,-3,22,1,0,1,0,0
3,10,2,1,-7,-8.0,12.0,41.0,21.0,-14,3,...,-2,-8,-29,29,7,0,0,1,0,0
4,1,-3,-2,-6,1.0,0.0,10.0,-6.0,4,0,...,-8,-11,-2,19,4,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2,1,0,-3,-17.0,-1.0,9.0,3.0,-7,-3,...,-4,-52,-41,-5,11,3,0,0,0,1
2996,2,5,7,13,17.0,62.0,5.0,7.0,-21,-12,...,-5,0,-17,-9,0,2,0,0,0,1
2997,-4,0,1,-1,4.0,56.0,6.0,6.0,9,2,...,-1,15,15,-1,-2,3,0,0,1,0
2998,18,-3,1,4,15.0,19.0,-1.0,-1.0,-10,8,...,-3,14,1,-3,-2,1,0,1,0,0


## Prepare data split

In [69]:
X_train = train_pd.drop('label', axis=1)
X_test = test_pd.drop('label', axis=1)
y_train = train_pd.label
y_test = test_pd.label

## Dimensionality reduction

In [70]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [71]:
dim_reduce = GaussianRandomProjection(n_components=10).fit(X_train)

In [72]:
X_train = dim_reduce.transform(X_train)

In [73]:
X_test = dim_reduce.transform(imp_mean.fit_transform(X_test))

## Create model

### Acurracy

In [74]:
tpot = TPOTClassifier(generations=2, population_size=10, 
                      n_jobs=-1, 
                      verbosity=2)
tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.5512773037191591
Generation 2 - Current best internal CV score: 0.5542639393806666

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.3, min_samples_leaf=1, min_samples_split=5, n_estimators=100)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=2,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=10,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [76]:
tpot.score(X_test, y_test)

0.9996666666666667

### F1-score

In [25]:
tpot = TPOTClassifier(generations=6, population_size=30, 
                      n_jobs=-1, 
                      scoring='f1_macro',
                      verbosity=2)
tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 1.0
Generation 2 - Current best internal CV score: 1.0
Generation 3 - Current best internal CV score: 1.0
Generation 4 - Current best internal CV score: 1.0
Generation 5 - Current best internal CV score: 1.0
Generation 6 - Current best internal CV score: 1.0

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.2, min_samples_leaf=8, min_samples_split=6, n_estimators=100)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=6,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=30,
               random_state=None, scoring='f1_macro', subsample=1.0,
               template=None, use_dask=False, verbosity=2, warm_start=False)

In [26]:
tpot.score(X_test, y_test)

Imputing missing values in feature set


0.24523022936082453